In [2]:
# source https://github.com/corynguyen19/midi-lstm-gan/blob/master/mlp_gan.py
# -------------------- imports --------------------------------
from __future__ import print_function, division

import os
import sys
import re
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, LSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.utils import np_utils

# Imports to work with https://colinraffel.com/projects/lmd/ - The Lakh MIDI Dataset v0.1
# The Lakh MIDI Dataset is distributed with a CC-BY 4.0 license;
# if you use this data in any capacity, please reference this page and my thesis:
# CC Colin Raffel. "Learning-Based Methods for Comparing Sequences,
# with Applications to Audio-to-MIDI Alignment and Matching". PhD Thesis, 2016.

%matplotlib inline
import pretty_midi
import librosa
import mir_eval
import mir_eval.display
import tables
import IPython.display
import json



DIR = './static/classical_midi_piano/'


Using TensorFlow backend.


# GAN
used with LSTM
source: https://github.com/corynguyen19/midi-lstm-gan/blob/master/mlp_gan.py

made some not significant changes

In [12]:
def get_notes():
    number_files = len(os.listdir(DIR))
    i = 0
    """ Get all the notes and chords from the midi files """
    notes = []
    print(' ======== getting notes ============')

    for file in os.listdir(DIR):
#         if filename.endswith(".asm") or filename.endswith(".py"): 
        i = i+1
        print(f'{i}/{number_files} - {file}')
        
        midi = converter.parse(os.path.join(DIR, file))

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [4]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)
    return (network_input, network_output)

In [5]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

In [6]:
# Load and convert the data
# =========== TAKES AROUND 10 minutes ========================
notes = get_notes()
n_vocab = len(set(notes))

 ======== getting notes ============
TRDVKOM128F92F90C4.midi
Parsing TRDVKOM128F92F90C4.midi
TRSQHIC128F92EE1F7.midi
Parsing TRSQHIC128F92EE1F7.midi
TRTTSVW128F92CA9EE.midi
Parsing TRTTSVW128F92CA9EE.midi
TRQKJSU128F42AEF49.midi
Parsing TRQKJSU128F42AEF49.midi
TRKKFJB128F934198B.midi
Parsing TRKKFJB128F934198B.midi
TRGJAGS128F92E22D9.midi
Parsing TRGJAGS128F92E22D9.midi
TRVGYXB128F9312D56.midi
Parsing TRVGYXB128F9312D56.midi
TRSMLHT128F4237D62.midi
Parsing TRSMLHT128F4237D62.midi
TROZGHV128F42861EE.midi
Parsing TROZGHV128F42861EE.midi
TRXTGHD128F425FF61.midi
Parsing TRXTGHD128F425FF61.midi
TRNCWCE128F4265233.midi
Parsing TRNCWCE128F4265233.midi
TRHQRWY128F148D7CA.midi
Parsing TRHQRWY128F148D7CA.midi
TRXBADZ128F92D1A03.midi
Parsing TRXBADZ128F92D1A03.midi
TRMWYTT128F426B33E.midi
Parsing TRMWYTT128F426B33E.midi
TRYQZRP128F424CA04.midi
Parsing TRYQZRP128F424CA04.midi
TRTJUTE12903CC4BBE.midi
Parsing TRTJUTE12903CC4BBE.midi
TRRWZSR12903CAC5B6.midi
Parsing TRRWZSR12903CAC5B6.midi
TRBMZIY128F

Parsing TREXKBK128F42A2C51.midi
TRMNUMV128F9309359.midi
Parsing TRMNUMV128F9309359.midi
TRFVMKU128F92E27EA.midi
Parsing TRFVMKU128F92E27EA.midi
TRXTHGK128EF341F20.midi
Parsing TRXTHGK128EF341F20.midi
TRRZBIT128F42A341E.midi
Parsing TRRZBIT128F42A341E.midi
TRHLNCH128F4272A0E.midi
Parsing TRHLNCH128F4272A0E.midi
TRLGSIL128F423CAA0.midi
Parsing TRLGSIL128F423CAA0.midi
TREKKQS128E07962F2.midi
Parsing TREKKQS128E07962F2.midi
TRVKXDF128F933C35F.midi
Parsing TRVKXDF128F933C35F.midi
TRPPHPQ128F4258FFC.midi
Parsing TRPPHPQ128F4258FFC.midi
TRUILNQ128F92F80EE.midi
Parsing TRUILNQ128F92F80EE.midi
TRWEWWW128F4285E21.midi
Parsing TRWEWWW128F4285E21.midi
TRRKXSD128F427DB15.midi
Parsing TRRKXSD128F427DB15.midi
TRSIIGM128F930EC04.midi
Parsing TRSIIGM128F930EC04.midi
TRJYDLV128F932C35D.midi
Parsing TRJYDLV128F932C35D.midi
TRJALFW128F1458672.midi
Parsing TRJALFW128F1458672.midi
TRXSTIT128F9318667.midi
Parsing TRXSTIT128F9318667.midi
TRCKQSE128F4277411.midi
Parsing TRCKQSE128F4277411.midi
TRVOVZV128EF367C

Parsing TRJTLTQ128F42852B5.midi
TRICDKG128F148AF50.midi
Parsing TRICDKG128F148AF50.midi
TRPVPXM128F148C4AB.midi
Parsing TRPVPXM128F148C4AB.midi
TRIZRWP128F933BD75.midi
Parsing TRIZRWP128F933BD75.midi
TRTTCXB12903CB27E2.midi
Parsing TRTTCXB12903CB27E2.midi
TRENSZA12903CAA8E5.midi
Parsing TRENSZA12903CAA8E5.midi
TROYJGC128F9316AF4.midi
Parsing TROYJGC128F9316AF4.midi
TRSOOHR128F93259C8.midi
Parsing TRSOOHR128F93259C8.midi
TRABGCD128F42884EB.midi
Parsing TRABGCD128F42884EB.midi
TRREALI12903CE9E41.midi
Parsing TRREALI12903CE9E41.midi
TRKIUWY128F932AD5D.midi
Parsing TRKIUWY128F932AD5D.midi
TRHSQAU128F146630D.midi
Parsing TRHSQAU128F146630D.midi
TRCRJXI128F425ACF2.midi
Parsing TRCRJXI128F425ACF2.midi
TRWRFXC128F427E1FB.midi
Parsing TRWRFXC128F427E1FB.midi
TRKYBQX128E0791657.midi
Parsing TRKYBQX128E0791657.midi
TRBCFKX128F1485FC2.midi
Parsing TRBCFKX128F1485FC2.midi
TRJLFDW128F42387F6.midi
Parsing TRJLFDW128F42387F6.midi
TRWDCWC128F147D427.midi
Parsing TRWDCWC128F147D427.midi
TRWFRQM128F93473

Parsing TRTLIGT128F1455D9E.midi
TRVSWQX12903CCB823.midi
Parsing TRVSWQX12903CCB823.midi
TRWGKYO128F930112B.midi
Parsing TRWGKYO128F930112B.midi
TRHMCSZ128F92FB59E.midi
Parsing TRHMCSZ128F92FB59E.midi
TRCWTHE128F4264D5C.midi
Parsing TRCWTHE128F4264D5C.midi
TRGOFSA128F4285C0F.midi
Parsing TRGOFSA128F4285C0F.midi
TRTSTMA128F14AEC4E.midi
Parsing TRTSTMA128F14AEC4E.midi
TROMEPT12903CE2637.midi
Parsing TROMEPT12903CE2637.midi
TRVKJFR128F14876D3.midi
Parsing TRVKJFR128F14876D3.midi
TRZMWZX128F930BECD.midi
Parsing TRZMWZX128F930BECD.midi
TRQODUP128F93446A2.midi
Parsing TRQODUP128F93446A2.midi
TROHBIJ128F14A588B.midi
Parsing TROHBIJ128F14A588B.midi
TRMJLBE128E0783E10.midi
Parsing TRMJLBE128E0783E10.midi
TRSASYZ12903D026BC.midi
Parsing TRSASYZ12903D026BC.midi
TRFBZFF128F425AE74.midi
Parsing TRFBZFF128F425AE74.midi
TRPCPLS128F4284909.midi
Parsing TRPCPLS128F4284909.midi
TRZTNXV12903D01126.midi
Parsing TRZTNXV12903D01126.midi
TRJAZUU128F42383B7.midi
Parsing TRJAZUU128F42383B7.midi
TRAUEUL128F93292

Parsing TRRBBXA128F147325B.midi
TRSZPKH128F423268C.midi
Parsing TRSZPKH128F423268C.midi
TRECCMJ128F932B4E8.midi
Parsing TRECCMJ128F932B4E8.midi
TRQOHTD12903C9E977.midi
Parsing TRQOHTD12903C9E977.midi
TRYYHEN128F9333658.midi
Parsing TRYYHEN128F9333658.midi
TRAMPQY128F9304A18.midi
Parsing TRAMPQY128F9304A18.midi
TRTVCMI128F425B742.midi
Parsing TRTVCMI128F425B742.midi
TRQXDAU128F42A19A6.midi
Parsing TRQXDAU128F42A19A6.midi
TRUITFK12903CB8681.midi
Parsing TRUITFK12903CB8681.midi
TRVSRRM128F4294EB7.midi
Parsing TRVSRRM128F4294EB7.midi
TRGSDLL128F9303452.midi
Parsing TRGSDLL128F9303452.midi
TRCWOYA128F4279D65.midi
Parsing TRCWOYA128F4279D65.midi
TRFAFKF128F92C64AB.midi
Parsing TRFAFKF128F92C64AB.midi
TRDQHSR128F147D40E.midi
Parsing TRDQHSR128F147D40E.midi
TRFTFKU128F9325519.midi
Parsing TRFTFKU128F9325519.midi
TRFUWBS12903D050BA.midi
Parsing TRFUWBS12903D050BA.midi
TRPVGJR128F93413CE.midi
Parsing TRPVGJR128F93413CE.midi
TRXXGYO128F4277EA3.midi
Parsing TRXXGYO128F4277EA3.midi
TRRLOPU12903C973

...

Parsing TRLZVVZ128F14524B2.midi
TRMMQZZ128F4249F40.midi
Parsing TRMMQZZ128F4249F40.midi
TRZVKAJ128F9320C03.midi
Parsing TRZVKAJ128F9320C03.midi
TRNYVES12903CB32B5.midi
Parsing TRNYVES12903CB32B5.midi
TRUNLHT128F1470AF0.midi
Parsing TRUNLHT128F1470AF0.midi
TRBKFBJ12903CF71F4.midi
Parsing TRBKFBJ12903CF71F4.midi
TRNAJHK12903CAC69B.midi
Parsing TRNAJHK12903CAC69B.midi
TRGJEWF12903CE3F6C.midi
Parsing TRGJEWF12903CE3F6C.midi
TRZOVLD12903CF4BB5.midi
Parsing TRZOVLD12903CF4BB5.midi
TRLVYLG128F9332650.midi
Parsing TRLVYLG128F9332650.midi
TRBBMRN128F42890CF.midi
Parsing TRBBMRN128F42890CF.midi
TRACEUX128F425CE5A.midi
Parsing TRACEUX128F425CE5A.midi
TRGIWWN128F1462742.midi
Parsing TRGIWWN128F1462742.midi
TRUCBFC128F92F750A.midi
Parsing TRUCBFC128F92F750A.midi
TRHFJDB128F92F0CC6.midi
Parsing TRHFJDB128F92F0CC6.midi
TRPAWAH128F428EA57.midi
Parsing TRPAWAH128F428EA57.midi
TRNCRKL128F148CAA7.midi
Parsing TRNCRKL128F148CAA7.midi
TRYNTUN128F933BD48.midi
Parsing TRYNTUN128F933BD48.midi
TRFQRTW128F42765

Parsing TRBREPT12903D01038.midi
TRVBSJT12903CAD42E.midi
Parsing TRVBSJT12903CAD42E.midi
TRBSAPP128F9344455.midi
Parsing TRBSAPP128F9344455.midi
TRNRKLE128F930F8FB.midi
Parsing TRNRKLE128F930F8FB.midi
TRLBDYR128F9302497.midi
Parsing TRLBDYR128F9302497.midi
TRATUVC128F42AD02E.midi
Parsing TRATUVC128F42AD02E.midi
TRJLDRE128F422B407.midi
Parsing TRJLDRE128F422B407.midi
TRECFQS128EF34B965.midi
Parsing TRECFQS128EF34B965.midi
TRVQVNW128F92CB769.midi
Parsing TRVQVNW128F92CB769.midi
TRVMXRF128F422F7E2.midi
Parsing TRVMXRF128F422F7E2.midi
TRHPHKT128F42961A7.midi
Parsing TRHPHKT128F42961A7.midi
TRNCDVJ12903CB5A7E.midi
Parsing TRNCDVJ12903CB5A7E.midi
TRFWQJL12903CEC341.midi
Parsing TRFWQJL12903CEC341.midi
TRVUOEX128F42433FC.midi
Parsing TRVUOEX128F42433FC.midi
TRFJIFX12903D050BC.midi
Parsing TRFJIFX12903D050BC.midi
TRVLEST128E078BB53.midi
Parsing TRVLEST128E078BB53.midi
TRLNMFO128F92C4254.midi
Parsing TRLNMFO128F92C4254.midi
TRJZWLO128F14636C5.midi
Parsing TRJZWLO128F14636C5.midi
TRSRXRF128F42967

Parsing TRKBPPK128F4269A25.midi
TRXNNLN128F92CBA7F.midi
Parsing TRXNNLN128F92CBA7F.midi
TRFPROV128F428C3B7.midi
Parsing TRFPROV128F428C3B7.midi
TRNTEAG128F14A141B.midi
Parsing TRNTEAG128F14A141B.midi
TRCDJOV128F93385AE.midi
Parsing TRCDJOV128F93385AE.midi
TRFUQYL12903CAD5AF.midi
Parsing TRFUQYL12903CAD5AF.midi
TRLFGUW128F425AD37.midi
Parsing TRLFGUW128F425AD37.midi
TRNCHOJ128E0786D00.midi
Parsing TRNCHOJ128E0786D00.midi
TRONMIY128E0795ABA.midi
Parsing TRONMIY128E0795ABA.midi
TRAQEDY128F1466365.midi
Parsing TRAQEDY128F1466365.midi
TRJQKMC128F422F7F5.midi
Parsing TRJQKMC128F422F7F5.midi
TRITPNN128F4279C0F.midi
Parsing TRITPNN128F4279C0F.midi
TRHCZJB128F9310D45.midi
Parsing TRHCZJB128F9310D45.midi
TRGCGEU128F148A282.midi
Parsing TRGCGEU128F148A282.midi
TRHSMSE128F92F291F.midi
Parsing TRHSMSE128F92F291F.midi
TRAIWKU128F9304D05.midi
Parsing TRAIWKU128F9304D05.midi
TRCKFSL128F92CBA6E.midi
Parsing TRCKFSL128F92CBA6E.midi
TROAMRR128F930ABCA.midi
Parsing TROAMRR128F930ABCA.midi
TRIJHGN12903CE6A

Parsing TRVGOUU128F934D0F7.midi
TRQEJQM128F42899B0.midi
Parsing TRQEJQM128F42899B0.midi
TRSALTB128F9310910.midi
Parsing TRSALTB128F9310910.midi
TRLQVBA12903CD5B49.midi
Parsing TRLQVBA12903CD5B49.midi
TRMCQVT12903CF16E5.midi
Parsing TRMCQVT12903CF16E5.midi
TRQRJLO128F9334A46.midi
Parsing TRQRJLO128F9334A46.midi
TRCSMCB12903D0A246.midi
Parsing TRCSMCB12903D0A246.midi
TRKBLFG128F148A1F1.midi
Parsing TRKBLFG128F148A1F1.midi
TRIKUEF128F148D7C6.midi
Parsing TRIKUEF128F148D7C6.midi
TRSIEZY12903CE5DB7.midi
Parsing TRSIEZY12903CE5DB7.midi
TRTKIEU128F92F0707.midi
Parsing TRTKIEU128F92F0707.midi
TREPTOI12903CD70E4.midi
Parsing TREPTOI12903CD70E4.midi
TREWIRZ128F1457FAD.midi
Parsing TREWIRZ128F1457FAD.midi
TRQOQCZ128F42672F5.midi
Parsing TRQOQCZ128F42672F5.midi
TRJNLCK128F1463E2B.midi
Parsing TRJNLCK128F1463E2B.midi
TRXNGER12903CC46F6.midi
Parsing TRXNGER12903CC46F6.midi
TRWDNIJ128E07967F2.midi
Parsing TRWDNIJ128E07967F2.midi
TRZHJKA128F931AFA9.midi
Parsing TRZHJKA128F931AFA9.midi
TREXKVS128F9318E

Parsing TRYYXHU128F4295F30.midi
TRGPHCV128F9351FA0.midi
Parsing TRGPHCV128F9351FA0.midi
TRZFUQT128F4266250.midi
Parsing TRZFUQT128F4266250.midi
TRWTWKY128F931A941.midi
Parsing TRWTWKY128F931A941.midi
TRCVKVN128F426D9CF.midi
Parsing TRCVKVN128F426D9CF.midi
TRPOWYV128F4296E52.midi
Parsing TRPOWYV128F4296E52.midi
TRFKYVR128F42791FD.midi
Parsing TRFKYVR128F42791FD.midi
TRNTCEJ12903CD6962.midi
Parsing TRNTCEJ12903CD6962.midi
TRSHFIW128E0783FB2.midi
Parsing TRSHFIW128E0783FB2.midi
TRLDFTF128F92FB5AD.midi
Parsing TRLDFTF128F92FB5AD.midi
TRJVSAC128F423CD6E.midi
Parsing TRJVSAC128F423CD6E.midi
TRYRMKC128F9320C1A.midi
Parsing TRYRMKC128F9320C1A.midi
TRRBTFX128F932AC61.midi
Parsing TRRBTFX128F932AC61.midi
TRZXRLH128E07878F3.midi
Parsing TRZXRLH128E07878F3.midi
TRROVGR12903CE9E33.midi
Parsing TRROVGR12903CE9E33.midi
TRLVPWB128F92D2D8F.midi
Parsing TRLVPWB128F92D2D8F.midi
TRZSECC128F1452788.midi
Parsing TRZSECC128F1452788.midi
TRYCBCH128E0796616.midi
Parsing TRYCBCH128E0796616.midi
TRSMOTK128F42851

In [14]:
class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):
        
        print(f'n_vocab {n_vocab}')
        
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.disc_loss.append(d_loss[0])
                self.gen_loss.append(g_loss)
                if epoch % 10 == 0:
                    self.generate(notes, f'./results/epoch_{epoch}')
        
        self.generate(notes, 'final')
        self.plot_loss()
        
    def generate(self, input_notes, filename):
        # Get pitch names and store in a dictionary
        notes = input_notes
        
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        
        tmp_notes = [((x+1)*(n_vocab-1)/2) for x in predictions[0]]
            
        pred_notes = [int_to_note[int(x)] for x in tmp_notes]
        
        
        create_midi(pred_notes, filename)
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()


In [16]:
EPOCHS = 20
gan = GAN(rows=100)    
gan.train(epochs=EPOCHS, batch_size=32, sample_interval=1)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1024)              4198400   
_________________________________________________________________
dense_15 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

/home/hany/MVI/__venv__/lib64/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/hany/MVI/__venv__/lib64/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.692045, acc.: 51.56%] [G loss: 0.688131]


/home/hany/MVI/__venv__/lib64/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.664324, acc.: 85.94%] [G loss: 0.686860]
2 [D loss: 0.632867, acc.: 76.56%] [G loss: 0.684220]
3 [D loss: 0.542174, acc.: 79.69%] [G loss: 0.673787]
4 [D loss: 0.453919, acc.: 76.56%] [G loss: 0.729772]
5 [D loss: 0.348894, acc.: 90.62%] [G loss: 0.784722]
6 [D loss: 0.395734, acc.: 84.38%] [G loss: 0.939699]
7 [D loss: 0.288757, acc.: 85.94%] [G loss: 1.195649]
8 [D loss: 0.283996, acc.: 92.19%] [G loss: 1.820673]
9 [D loss: 0.224411, acc.: 93.75%] [G loss: 3.035042]
10 [D loss: 0.304430, acc.: 87.50%] [G loss: 1.533074]
11 [D loss: 0.257522, acc.: 93.75%] [G loss: 1.512250]
12 [D loss: 0.248157, acc.: 96.88%] [G loss: 2.951439]
13 [D loss: 0.287961, acc.: 89.06%] [G loss: 2.379136]
14 [D loss: 0.443825, acc.: 87.50%] [G loss: 1.574619]
15 [D loss: 0.345551, acc.: 89.06%] [G loss: 1.756889]
16 [D loss: 0.355513, acc.: 89.06%] [G loss: 2.072107]
17 [D loss: 0.234578, acc.: 93.75%] [G loss: 2.391032]
18 [D loss: 0.377844, acc.: 90.62%] [G loss: 1.995554]
19 [D loss: 0.44530

In [18]:
gan.combined.save(f'final{EPOCHS}model.h5')

/home/hany/MVI/__venv__/lib64/python3.7/site-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '
